In [ ]:
import pandas as pd
import os

def split_annotations(input_csv, directory1, directory2):
    annotations_df = pd.read_csv(input_csv)
    files_dir1 = os.listdir(directory1)
    files_dir2 = os.listdir(directory2)

    print("arquivos do diretório 1:")
    print(files_dir1)
    print("arquivos do diretório 2:")
    print(files_dir2)

    annotations_dir1 = annotations_df[annotations_df['img_name'].isin(files_dir1)]
    annotations_dir2 = annotations_df[annotations_df['img_name'].isin(files_dir2)]

    print("anotações do 1:")
    print(annotations_dir1)
    print("anotações do 2:")
    print(annotations_dir2)

    annotations_dir1.to_csv('test_label_filt.csv', index=False)
    annotations_dir2.to_csv('train_label_filt.csv', index=False)

directory1 = 'dataset/test/images'
directory2 = 'dataset/train/images'

input_csv = 'super-annotation.csv'

split_annotations(input_csv, directory1, directory2)

In [ ]:
input_csv = 'train_label_filt.csv'
output_csv1 = 'train_label_filt2.csv'
output_csv2 = 'train_label_filt_reorganized.csv'

df = pd.read_csv(input_csv)
df.to_csv(output_csv1, index=False)

df_reorganized = df[['label', 'x', 'y', 'width', 'height']]
df_reorganized['img_name'] = df['img_name']
df_reorganized = df_reorganized[['label', 'x', 'y', 'width', 'height', 'img_name']]

df_reorganized.to_csv(output_csv2, index=False)

In [ ]:
input_dir = ''
csv_filename = 'test_label_filt_reorganized.csv'

output_dir = 'dataset/test/labels'
os.makedirs(output_dir, exist_ok=True)
df = pd.read_csv(os.path.join(input_dir, csv_filename))

for index, row in df.iterrows():
    txt_filename = row['img_name'].split('.')[0] + '.txt'
    txt_content = f"{row['label']} {row['x']} {row['y']} {row['width']} {row['height']}"

    with open(os.path.join(output_dir, txt_filename), 'w') as txt_file:
        txt_file.write(txt_content)

In [ ]:
pasta_txt = '/content/drive/MyDrive/final_code/dataset/train/labels'

for nome_arquivo in os.listdir(pasta_txt):
    if nome_arquivo.endswith('.txt'):
        caminho_arquivo = os.path.join(pasta_txt, nome_arquivo)

        with open(caminho_arquivo, 'r') as arquivo:
            linhas = arquivo.readlines()

        with open(caminho_arquivo, 'w') as arquivo:
            for linha in linhas:
                partes = linha.split()
                if partes:
                    if partes[0] == '1':
                        partes[0] = '0'
                    arquivo.write(' '.join(partes) + '\n')

In [ ]:
caminho_pasta = '/content/drive/MyDrive/final_code/dataset/train/labels'

largura_imagem = 512
altura_imagem = 512

arquivos = os.listdir(caminho_pasta)

for nome_arquivo in arquivos:
    if nome_arquivo.endswith('.txt'):
        caminho_arquivo = os.path.join(caminho_pasta, nome_arquivo)

        with open(caminho_arquivo, 'r') as arquivo:
            linhas = arquivo.readlines()
        with open(caminho_arquivo, 'w') as arquivo:
            for linha in linhas:
                partes = linha.strip().split(' ')
                if len(partes) >= 5:
                    classe = partes[0]
                    x = float(partes[1]) / largura_imagem
                    y = float(partes[2]) / altura_imagem
                    largura = float(partes[3]) / largura_imagem
                    altura = float(partes[4]) / altura_imagem
                    arquivo.write(f'{classe} {x} {y} {largura} {altura}\n')

        print(f'Coordenadas normalizadas escritas em {caminho_arquivo}')

print('Conversão para coordenadas normalizadas concluída.')

In [1]:
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Pip install method (recommended)
!pip install --upgrade pip
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.1/720.1 kB 14.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.25 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.4/201.2 GB disk)


In [5]:
from ultralytics import YOLO

from IPython.display import display, Image

In [7]:
os.chdir('/content/drive/MyDrive/final_code')

In [8]:
!apt-get update
!apt-get install libgl1-mesa-glx -y

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,076 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,843 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restr

In [10]:
%%writefile data.yaml
train: "/content/drive/MyDrive/lung_nodule/dataset/train"
val: "/content/drive/MyDrive/lung_nodule/dataset/test"

nc: 1  # number of classes
names: ['Nodule']

Overwriting data.yaml


In [13]:
from ultralytics import RTDETR
model = RTDETR(model='rtdetr-l.pt')
result=model.train(
        name='RT-DETR_001',
        epochs=60,
        imgsz=640,
        batch=4,
        optimizer='SGD',
        lr0=0.0005,
        lrf=0.01,
        # augmentation
        close_mosaic=0,
        # hsv_h=0,
        # hsv_s=0,
        # hsv_v=0,
        # degrees=0,
        # translate=0,
        # scale=0,
        # shear=0,
        # perspective=0,
        # flipud=0,
        # fliplr=0,
        mosaic=1,
        mixup=0,
        data='data.yaml',
    )

Ultralytics YOLOv8.1.25 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=data.yaml, epochs=60, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=RT-DETR_0014, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /content/drive/MyDrive/lung_nodule/dataset/train/labels.cache... 3342 images, 16 backgrounds, 0 corrupt: 100%|██████████| 3358/3358 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/lung_nodule/dataset/test/labels.cache... 928 images, 7 backgrounds, 0 corrupt: 100%|██████████| 935/935 [00:00<?, ?it/s]


Plotting labels to runs/detect/RT-DETR_0014/labels.jpg... 
optimizer: SGD(lr=0.0005, momentum=0.937) with parameter groups 143 weight(decay=0.0), 206 weight(decay=0.0005), 226 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/RT-DETR_0014
Starting training for 60 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

                   all        935        928      0.556      0.559      0.526       0.23

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/840 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:251: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:71.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
      60/60      4.08G     0.6733     0.6594     0.1219          2        640: 100%|██████████| 840/840 [05:03<00:00,  2.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 117/117 [00:19<00:00,  6.16it/s]


                   all        935        928      0.547       0.56      0.523      0.228

60 epochs completed in 5.384 hours.
Optimizer stripped from runs/detect/RT-DETR_0014/weights/last.pt, 66.1MB
Optimizer stripped from runs/detect/RT-DETR_0014/weights/best.pt, 66.1MB

Validating runs/detect/RT-DETR_0014/weights/best.pt...
Ultralytics YOLOv8.1.25 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
rt-detr-l summary: 498 layers, 31985795 parameters, 0 gradients, 103.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 117/117 [00:18<00:00,  6.48it/s]


                   all        935        928       0.56      0.551       0.53      0.234
Speed: 0.2ms preprocess, 15.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/RT-DETR_0014
